In [1]:
import requests
import json
import pandas as pd
import pickle
import re
from sqlalchemy import create_engine
import pymysql

### 챔피언데이터 불러오기

In [2]:
with open('championList.pickle','rb') as f:
    datas = pickle.load(f)

### 문자형 데이터 변환 함수작성

In [3]:
def rename(st):
    txt = re.compile('[^ \u3131-\u3163\uac00-\ud7a3-1-9%.?]+')
    result = txt.sub('',st)
    result = result.replace('  ',' ?')
    result = result.replace('? ','?')
    result = result.replace('?1 ','?')
    result = result[:-1]
    return result

### 데이터 가공 및 변환

In [4]:
champion = []
for data in datas["data"]:
    champion.append(data)

champInfo, passiveP = [],[]
for champ in champion:
    url = requests.get('http://ddragon.leagueoflegends.com/cdn/12.19.1/data/ko_KR/champion/'+ champ +'.json')
    urlInfo = url.json()['data']
    champInfo.append(urlInfo)

key, name, en_name, image, passive, spellQ, spellW, spellE, spellR = [],[],[],[],[],[],[],[],[]
for result, i in zip(champInfo, champion):
    key.append([result[i]['key']])
    name.append([result[i]['name']])
    en_name.append([result[i]['id']])
    passive.append([result[i]['passive']['name'],rename(result[i]['passive']['description'])
                    ,'https://ddragon.leagueoflegends.com/cdn/12.19.1/img/passive/'+result[i]['passive']['image']['full']])
    image.append(["http://ddragon.leagueoflegends.com/cdn/12.19.1/img/champion/" + result[i]['image']['full']])
    spellQ.append([result[i]['spells'][0]['name'],"https://ddragon.leagueoflegends.com/cdn/12.19.1/img/spell/"+result[i]['spells'][0]['id']+'.png'
          ,result[i]['spells'][0]['cooldownBurn'],result[i]['spells'][0]['costBurn'],result[i]['spells'][0]['rangeBurn']
          ,rename(result[i]['spells'][0]['description']),rename(result[i]['spells'][0]['tooltip'])])
    spellW.append([result[i]['spells'][1]['name'],"https://ddragon.leagueoflegends.com/cdn/12.19.1/img/spell/"+result[i]['spells'][1]['id']+'.png'
          ,result[i]['spells'][1]['cooldownBurn'],result[i]['spells'][1]['costBurn'],result[i]['spells'][1]['rangeBurn']
          ,rename(result[i]['spells'][1]['description']),rename(result[i]['spells'][1]['tooltip'])])
    spellE.append([result[i]['spells'][2]['name'],"https://ddragon.leagueoflegends.com/cdn/12.19.1/img/spell/"+result[i]['spells'][2]['id']+'.png'
          ,result[i]['spells'][2]['cooldownBurn'],result[i]['spells'][2]['costBurn'],result[i]['spells'][2]['rangeBurn']
          ,rename(result[i]['spells'][2]['description']),rename(result[i]['spells'][2]['tooltip'])])
    spellR.append([result[i]['spells'][3]['name'],"https://ddragon.leagueoflegends.com/cdn/12.19.1/img/spell/"+result[i]['spells'][3]['id']+'.png'
          ,result[i]['spells'][3]['cooldownBurn'],result[i]['spells'][3]['costBurn'],result[i]['spells'][3]['rangeBurn']
          ,rename(result[i]['spells'][3]['description']),rename(result[i]['spells'][3]['tooltip'])])

df1 = list(map(list.__add__,key,name))
df1 = list(map(list.__add__,df1,en_name))
df1 = list(map(list.__add__,df1,image))
df1 = list(map(list.__add__,df1,passive))
df1 = list(map(list.__add__,df1,spellQ))
df1 = list(map(list.__add__,df1,spellW))
df1 = list(map(list.__add__,df1,spellE))
df1 = list(map(list.__add__,df1,spellR))
df = pd.DataFrame(df1, columns=['champion_key','name','en_name','portrait'
                                ,'pName','pDst','pImage'
                                ,'qName','qImage','qCooldown','qCost','qRange','qDst','qTip'
                                ,'wName','wImage','wCooldown','wCost','wRange','wDst','wTip'
                                ,'eName','eImage','eCooldown','eCost','eRange','eDst','eTip'
                                ,'rName','rImage','rCooldown','rCost','rRange','rDst','rTip'])

In [5]:
df

,champion_key,name,en_name,portrait,pName,pDst,pImage,qName,qImage,qCooldown,...,eRange,eDst,eTip,rName,rImage,rCooldown,rCost,rRange,rDst,rTip
0,266,아트록스,Aatrox,http://ddragon.leagueoflegends.com/cdn/12.19.1...,사신 태세,주기적으로 아트록스의 기본 공격이 대상 최대 체력에 비례하여 추가 물리 피해를 입히...,https://ddragon.leagueoflegends.com/cdn/12.19....,다르킨의 검,https://ddragon.leagueoflegends.com/cdn/12.19....,14/12/10/8/6,...,25000,기본 지속 효과로 아트록스가 적 챔피언에게 피해를 입히면 체력을 회복합니다. 사용 ...,기본 지속 효과 아트록스가 챔피언을 상대로 ?%의 모든 피해 흡혈을 얻고 세계의 종...,세계의 종결자,https://ddragon.leagueoflegends.com/cdn/12.19....,120/100/80,0,25000,아트록스가 악마의 힘을 해방하여 근처 적 미니언에게 공포를 주고 자신의 공격력과 체...,아트록스가 진정한 악마의 모습을 드러내 근처 미니언이 ?초 동안 공포에 떨게 하고 ...
1,103,아리,Ahri,http://ddragon.leagueoflegends.com/cdn/12.19.1...,정기 흡수,아리가 미니언 또는 몬스터를 9마리 처치하면 체력을 회복합니다.아리가 적 챔피언 처...,https://ddragon.leagueoflegends.com/cdn/12.19....,현혹의 구슬,https://ddragon.leagueoflegends.com/cdn/12.19....,7,...,975,아리가 입맞춤을 날려 피해를 주며 맞은 적을 홀립니다. 홀린 적은 즉시 이동 스킬을...,아리가 입맞춤을 날려 첫 번째로 맞는 적을 ?초 동안 매혹하고 ?의 마법 피해를 입...,혼령 질주,https://ddragon.leagueoflegends.com/cdn/12.19....,140/115/90,100,450,아리가 전방으로 질주하며 근처 적 챔피언들에게 혼령의 정기를 쏘아냅니다. 재사용 대...,아리가 민첩하게 질주하며 근처 적챔피언 우선에게 혼령의 정기 ?개를 쏘아내 정기 하...
2,84,아칼리,Akali,http://ddragon.leagueoflegends.com/cdn/12.19.1...,암살자의 표식,스킬 공격으로 챔피언에게 피해를 입히면 해당 챔피언의 주변에 원이 생깁니다. 이 원...,https://ddragon.leagueoflegends.com/cdn/12.19....,오연투척검,https://ddragon.leagueoflegends.com/cdn/12.19....,1.5,...,825,뒤로 공중제비를 돌며 전방으로 표창을 던져 마법 피해를 입힙니다. 처음으로 적중한 ...,아칼리가 뒤로 공중제비를 돌며 표창을 던져 ?의 마법 피해를 입히고 표창에 맞은 첫...,무결처형,https://ddragon.leagueoflegends.com/cdn/12.19....,100/80/60,0,675,아칼리가 지정한 방향으로 도약해 공격이 적중한 적들에게 피해를 입힙니다. 재사용 시...,아칼리가 적 챔피언을 뛰어넘어 경로 내에 있는 모든 적에게 ?의 마법 피해를 입힙니...
3,166,아크샨,Akshan,http://ddragon.leagueoflegends.com/cdn/12.19.1...,비열한 싸움,아크샨의 기본 공격 및 스킬이 세 번 적중할 때마다 추가 물리 피해를 입힙니다. 대...,https://ddragon.leagueoflegends.com/cdn/12.19....,복수의 부메랑,https://ddragon.leagueoflegends.com/cdn/12.19....,8/7.25/6.5/5.75/5,...,800,아크샨이 갈고리총을 발사해 지형에 고정시킨 후 갈고리에 매달린 채 가장 가까운 적을...,첫 사용 시 아크샨이 적중한 첫 번째 지형에 고정되는 갈고리총을 발사합니다. ?두 ...,인과응보,https://ddragon.leagueoflegends.com/cdn/12.19....,100/85/70,100,2500,아크샨이 적 챔피언을 조준하고 탄환을 저장합니다. 사격을 개시하면 저장한 탄환을 모...,아크샨이 챔피언을 조준하고 최대 ?초간 총을 과충전해 탄환을 ?개까지 저장합니다. ...
4,12,알리스타,Alistar,http://ddragon.leagueoflegends.com/cdn/12.19.1...,승리의 포효,근처에서 적이 죽거나 알리스타가 적 챔피언을 기절 혹은 이동시킬 때마다 포효를 중첩...,https://ddragon.leagueoflegends.com/cdn/12.19....,분쇄,https://ddragon.leagueoflegends.com/cdn/12.19....,14/13/12/11/10,...,350,알리스타가 근처의 적 유닛을 짓밟아 적 챔피언에게 피해를 입힐 경우 중첩을 얻고 유...,알리스타가 땅을 짓밟기 시작하며 유체화 상태가 되고 ?3 초에 걸쳐 근처 적에게 ?...,꺾을 수 없는 의지,https://ddragon.leagueoflegends.com/cdn/12.19....,120/100/80,100,1,알리스타가 거친 함성을 지르며 자신에게 걸린 모든 방해 효과를 제거하고 일정 시간 ...,알리스타가 즉시 모든 방해 효과를 없애고 ?초 동안 받는 피해가 ?% 감소합니다.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,221,제리,Zeri,http://ddragon.leagueoflegends.com/cdn/12.19.1...,살아있는 배터리,제리가 보호막을 받을 때마다 이동 속도가 증가합니다. 제리가 적 보호막에 피해를 입...,https://ddragon.leagueoflegends.com/cdn/12.19....,집중 사격,https://ddragon.leagueoflegends.com/cdn/12.19....,0,...,25000,제리가 짧은 거리를 돌진하며 맞닥트리는 지형을 모두 뛰어넘습니다. 집중 사격의 다음...,제리가 짧은 거리를 돌진하며 맞닥트리는 지형을 모두 뛰어넘습니다. 지형을 뛰어넘으면...,번개 방출,https://ddragon.leagueoflegends.com/cdn/12.19....,100/85/70,100,800,제리가 전류를 방출하고 자신을 과충전해 피해량과 이동 속도가 증가합니다. 제리가 적...,제리가 전류를 방출해 근처 적에게 ?의 마법 피해를 입히고 ?초 동안 적중한 챔피언...
157,115,직스,Ziggs,http://ddragon.leagueoflegends.com/cdn/12.19.1...,짧은 도화선,주기적으로 직스의 다음 기본 공격이 추가 마법 피해를 입힙니다. 직스가 스킬을 사용...,https://ddragon.leagueoflegends.com/cdn/12.19....,반동 폭탄,https://ddragon.leagueoflegends.com/cdn/12.19....,6/5.5/5/4.5/4,...,900,직스가 밟으면 폭발하는 지뢰를 뿌려 지뢰에 닿은 적은 마법 피해를 입고 둔화됩니다....,직스가 밟으면 터지는 지뢰를 뿌려 지뢰에 닿은 적에게 ?의 마법 피해를 입히고 ?4...,지옥 화염 폭탄,https://ddragon.leagueoflegends.com/cdn/12.19....,120/95/70,100,5000,직스가 궁극의 무기 지옥 화염 폭탄을 조립하여 아주 멀리 날립니다. 폭발의 중심부에...,직스가 궁극의 무기를 던져 폭발 범위 중앙에 ?의 마법 피해를 입힙니다. 가장자리에...
158,26,질리언,Zilean,http://ddragon.leagueoflegends.com/cdn/12.19.1...,시간의 유리병,질리언은 시간을 경험치의 형태로 보존하여 아군에게 줄 수 있

In [6]:
df.loc[0,'pImage']

'https://ddragon.leagueoflegends.com/cdn/12.19.1/img/passive/Aatrox_Passive.png'

### DB에 저장하기

In [166]:
conn = pymysql.connect(host='3.36.255.55', user='root', password='root1234', db='lol_project', charset='utf8')
engine = create_engine('mysql+pymysql://{user}:{pw}@{host}/{db}'.format(user='root',pw='root1234',db='lol_project',host='3.36.255.55'))
conn = engine.connect()
df.to_sql(name='champion', con=engine ,if_exists='append',index=False)
conn.close()

### 피클에 저장하기

In [167]:
with open('championData.pickle','wb') as f:
    pickle.dump(df,f)

In [2]:
with open('championData.pickle','rb') as f:
    df = pickle.load(f)

In [3]:
df

,champion_key,name,en_name,portrait,pName,pDst,pImage,qName,qImage,qCooldown,...,eRange,eDst,eTip,rName,rImage,rCooldown,rCost,rRange,rDst,rTip
0,266,아트록스,Aatrox,http://ddragon.leagueoflegends.com/cdn/12.19.1...,사신 태세,주기적으로 아트록스의 기본 공격이 대상 최대 체력에 비례하여 추가 물리 피해를 입히...,https://ddragon.leagueoflegends.com/cdn/12.19....,다르킨의 검,https://ddragon.leagueoflegends.com/cdn/12.19....,14/12/10/8/6,...,25000,기본 지속 효과로 아트록스가 적 챔피언에게 피해를 입히면 체력을 회복합니다. 사용 ...,기본 지속 효과 아트록스가 챔피언을 상대로 ?%의 모든 피해 흡혈을 얻고 세계의 종...,세계의 종결자,https://ddragon.leagueoflegends.com/cdn/12.19....,120/100/80,0,25000,아트록스가 악마의 힘을 해방하여 근처 적 미니언에게 공포를 주고 자신의 공격력과 체...,아트록스가 진정한 악마의 모습을 드러내 근처 미니언이 ?초 동안 공포에 떨게 하고 ...
1,103,아리,Ahri,http://ddragon.leagueoflegends.com/cdn/12.19.1...,정기 흡수,아리가 미니언 또는 몬스터를 9마리 처치하면 체력을 회복합니다.아리가 적 챔피언 처...,https://ddragon.leagueoflegends.com/cdn/12.19....,현혹의 구슬,https://ddragon.leagueoflegends.com/cdn/12.19....,7,...,975,아리가 입맞춤을 날려 피해를 주며 맞은 적을 홀립니다. 홀린 적은 즉시 이동 스킬을...,아리가 입맞춤을 날려 첫 번째로 맞는 적을 ?초 동안 매혹하고 ?의 마법 피해를 입...,혼령 질주,https://ddragon.leagueoflegends.com/cdn/12.19....,140/115/90,100,450,아리가 전방으로 질주하며 근처 적 챔피언들에게 혼령의 정기를 쏘아냅니다. 재사용 대...,아리가 민첩하게 질주하며 근처 적챔피언 우선에게 혼령의 정기 ?개를 쏘아내 정기 하...
2,84,아칼리,Akali,http://ddragon.leagueoflegends.com/cdn/12.19.1...,암살자의 표식,스킬 공격으로 챔피언에게 피해를 입히면 해당 챔피언의 주변에 원이 생깁니다. 이 원...,https://ddragon.leagueoflegends.com/cdn/12.19....,오연투척검,https://ddragon.leagueoflegends.com/cdn/12.19....,1.5,...,825,뒤로 공중제비를 돌며 전방으로 표창을 던져 마법 피해를 입힙니다. 처음으로 적중한 ...,아칼리가 뒤로 공중제비를 돌며 표창을 던져 ?의 마법 피해를 입히고 표창에 맞은 첫...,무결처형,https://ddragon.leagueoflegends.com/cdn/12.19....,100/80/60,0,675,아칼리가 지정한 방향으로 도약해 공격이 적중한 적들에게 피해를 입힙니다. 재사용 시...,아칼리가 적 챔피언을 뛰어넘어 경로 내에 있는 모든 적에게 ?의 마법 피해를 입힙니...
3,166,아크샨,Akshan,http://ddragon.leagueoflegends.com/cdn/12.19.1...,비열한 싸움,아크샨의 기본 공격 및 스킬이 세 번 적중할 때마다 추가 물리 피해를 입힙니다. 대...,https://ddragon.leagueoflegends.com/cdn/12.19....,복수의 부메랑,https://ddragon.leagueoflegends.com/cdn/12.19....,8/7.25/6.5/5.75/5,...,800,아크샨이 갈고리총을 발사해 지형에 고정시킨 후 갈고리에 매달린 채 가장 가까운 적을...,첫 사용 시 아크샨이 적중한 첫 번째 지형에 고정되는 갈고리총을 발사합니다. ?두 ...,인과응보,https://ddragon.leagueoflegends.com/cdn/12.19....,100/85/70,100,2500,아크샨이 적 챔피언을 조준하고 탄환을 저장합니다. 사격을 개시하면 저장한 탄환을 모...,아크샨이 챔피언을 조준하고 최대 ?초간 총을 과충전해 탄환을 ?개까지 저장합니다. ...
4,12,알리스타,Alistar,http://ddragon.leagueoflegends.com/cdn/12.19.1...,승리의 포효,근처에서 적이 죽거나 알리스타가 적 챔피언을 기절 혹은 이동시킬 때마다 포효를 중첩...,https://ddragon.leagueoflegends.com/cdn/12.19....,분쇄,https://ddragon.leagueoflegends.com/cdn/12.19....,14/13/12/11/10,...,350,알리스타가 근처의 적 유닛을 짓밟아 적 챔피언에게 피해를 입힐 경우 중첩을 얻고 유...,알리스타가 땅을 짓밟기 시작하며 유체화 상태가 되고 ?3 초에 걸쳐 근처 적에게 ?...,꺾을 수 없는 의지,https://ddragon.leagueoflegends.com/cdn/12.19....,120/100/80,100,1,알리스타가 거친 함성을 지르며 자신에게 걸린 모든 방해 효과를 제거하고 일정 시간 ...,알리스타가 즉시 모든 방해 효과를 없애고 ?초 동안 받는 피해가 ?% 감소합니다.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,221,제리,Zeri,http://ddragon.leagueoflegends.com/cdn/12.19.1...,살아있는 배터리,제리가 보호막을 받을 때마다 이동 속도가 증가합니다. 제리가 적 보호막에 피해를 입...,https://ddragon.leagueoflegends.com/cdn/12.19....,집중 사격,https://ddragon.leagueoflegends.com/cdn/12.19....,0,...,25000,제리가 짧은 거리를 돌진하며 맞닥트리는 지형을 모두 뛰어넘습니다. 집중 사격의 다음...,제리가 짧은 거리를 돌진하며 맞닥트리는 지형을 모두 뛰어넘습니다. 지형을 뛰어넘으면...,번개 방출,https://ddragon.leagueoflegends.com/cdn/12.19....,100/85/70,100,800,제리가 전류를 방출하고 자신을 과충전해 피해량과 이동 속도가 증가합니다. 제리가 적...,제리가 전류를 방출해 근처 적에게 ?의 마법 피해를 입히고 ?초 동안 적중한 챔피언...
157,115,직스,Ziggs,http://ddragon.leagueoflegends.com/cdn/12.19.1...,짧은 도화선,주기적으로 직스의 다음 기본 공격이 추가 마법 피해를 입힙니다. 직스가 스킬을 사용...,https://ddragon.leagueoflegends.com/cdn/12.19....,반동 폭탄,https://ddragon.leagueoflegends.com/cdn/12.19....,6/5.5/5/4.5/4,...,900,직스가 밟으면 폭발하는 지뢰를 뿌려 지뢰에 닿은 적은 마법 피해를 입고 둔화됩니다....,직스가 밟으면 터지는 지뢰를 뿌려 지뢰에 닿은 적에게 ?의 마법 피해를 입히고 ?4...,지옥 화염 폭탄,https://ddragon.leagueoflegends.com/cdn/12.19....,120/95/70,100,5000,직스가 궁극의 무기 지옥 화염 폭탄을 조립하여 아주 멀리 날립니다. 폭발의 중심부에...,직스가 궁극의 무기를 던져 폭발 범위 중앙에 ?의 마법 피해를 입힙니다. 가장자리에...
158,26,질리언,Zilean,http://ddragon.leagueoflegends.com/cdn/12.19.1...,시간의 유리병,질리언은 시간을 경험치의 형태로 보존하여 아군에게 줄 수 있